In [37]:
from __future__ import annotations as _annotations

import numpy as np
from website_to_rag.agents.webcrawl_agent import search_agent
from website_to_rag.utils.webpage_extractor import embed_webpage, WebPage, Section
from typing import Set
from pydantic_graph import BaseNode, Edge, End, Graph, GraphRunContext, HistoryStep
from pydantic_ai import Agent, RunContext
from pydantic_ai.format_as_xml import format_as_xml
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass, field
from rich.prompt import Prompt

In [38]:
@dataclass
class State:
    user_query: str
    retrieval_tries: int = 0
    retrieval_agent_history: list[ModelMessage] = field(default_factory=set)
    unused_urls: Set[str] = field(default_factory=set)
    searched_urls: Set[str] = field(default_factory=set)
    unembedded_webpages: list[WebPage] = field(default_factory=list)
    embedded_webpages: list[tuple(WebPage, Section, list[list[float]])] = field(default_factory=list)

@dataclass
class Ask(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State]) -> Retrieval:
        ctx.state.user_query = Prompt.ask("What can I help you with?")
        return Retrieval

@dataclass
class RetrievalResult:
    top_5_results: list[str]

@dataclass
class NoRelevantData:
    ...

@dataclass
class RetrievalDeps:
    embedded_webpages: list[tuple(WebPage, Section, list[list[float]])]

retrieval_agent = Agent[RetrievalDeps, RetrievalResult|NoRelevantData](
    'openai:gpt-4o-mini',
    result_type=RetrievalResult|NoRelevantData,
    deps_type=RetrievalDeps,
    system_prompt=(
        "Given a user query use the `retrieval_tool` to search websites in memory for information relevant to answer the user's question. "
        "If the information recieved is not relevant to the user's query try changing the prompt you give to the retrieval tool. "
        "If there still doesn't appear to be relevant information return 'NoRelevantData'"
    ) 
)

@retrieval_agent.tool
async def retrieval_tool(ctx: RunContext[RetrievalDeps], prompt):
    ...